In [1]:
from nn import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

In [4]:
X_train, y_train = train_df.iloc[:, 1:], train_df.iloc[:, 0]
X_test, y_test = test_df.iloc[:, 1:], test_df.iloc[:, 0]

In [5]:
from tools import normalize

In [6]:
X_test = (X_test - X_train.mean()) / X_train.std()
X_train = (X_train - X_train.mean()) / X_train.std()

In [7]:
y_map = {"B": 0, "M":1}
y_train = y_train.map(y_map).values[:, np.newaxis].astype(np.float64)
y_test = y_test.map(y_map).values[:, np.newaxis].astype(np.float64)

In [8]:
def build_model(activation):
    nn = NeuralNetwork(loss='binary_crossentropy', verbose=False, verbose_step=25)
    nn.add(Dense(30, activation=activation))
    nn.add(Dense(65, activation=activation))
    nn.add(Dense(1, activation='sigmoid'))
    return nn

In [9]:
from metrics import accuracy

In [10]:
# %load_ext Cython

In [11]:
# %%cython --annotate
# import numpy as np
# cimport numpy as np
# cimport cython

# from libc.math cimport exp 

# @cython.cdivision(True)
# @cython.boundscheck(False)
# @cython.nonecheck(False)
# @cython.wraparound(False)
# cdef np.ndarray[np.double_t, ndim=2] sig(np.ndarray[np.double_t, ndim=2] x):
#     return -x

In [12]:
n_episodes = 5
activations = ['sigmoid', 'tanh', 'relu', 'leaky_relu', 'elu']
for activation in activations:
    scores = []
    np.random.seed(42)
    for ep in range(n_episodes):
        nn = build_model(activation)
        nn.fit(X_train.values, y_train, n_epochs=500, metrics=['acc'])
        acc_train = accuracy()(y_train, nn.predict(X_train.values) > .5)
        acc_test = accuracy()(y_test, nn.predict(X_test.values) > .5)
        scores.append([acc_train, acc_test])
    scores = np.array(scores)
    print(scores)
    mean_train_acc, mean_test_acc = scores.mean(axis=0)
    print(f"activation={activation} | mean_train_acc={mean_train_acc:.5f} | mean_test_acc={mean_test_acc:.5f}")

[[0.64553991 0.57342657]
 [0.64553991 0.57342657]
 [0.64553991 0.57342657]
 [0.64553991 0.57342657]
 [0.64553991 0.57342657]]
activation=sigmoid | mean_train_acc=0.64554 | mean_test_acc=0.57343
[[0.99061033 0.95804196]
 [0.99061033 0.95804196]
 [0.99061033 0.95804196]
 [0.99061033 0.95804196]
 [0.99061033 0.95104895]]
activation=tanh | mean_train_acc=0.99061 | mean_test_acc=0.95664
[[0.9600939  0.93006993]
 [0.95774648 0.93006993]
 [0.95774648 0.93006993]
 [0.83098592 0.77622378]
 [0.64553991 0.57342657]]
activation=relu | mean_train_acc=0.87042 | mean_test_acc=0.82797
[[0.98122066 0.95104895]
 [0.98122066 0.93706294]
 [0.98122066 0.95104895]
 [0.9741784  0.95104895]
 [0.95305164 0.93706294]]
activation=leaky_relu | mean_train_acc=0.97418 | mean_test_acc=0.94545
[[0.99061033 0.95804196]
 [0.99061033 0.95804196]
 [0.99061033 0.95804196]
 [0.99061033 0.95104895]
 [0.99061033 0.95804196]]
activation=elu | mean_train_acc=0.99061 | mean_test_acc=0.95664


In [13]:
nn = NeuralNetwork('binary_crossentropy', verbose=True, verbose_step=100, learning_rate=0.05, debug=False)
nn.add(Dense(30, activation='tanh'))
nn.add(Dense(65, activation='elu'))
nn.add(Dense(64, activation='leaky_relu'))
nn.add(Dense(1, activation='sigmoid'))

nn.fit(X_train.values, y_train, X_test.values, y_test, n_epochs=150, metrics=['acc'])


[100/150]: loss=0.65402 val_loss=0.68410 acc=0.64554 val_acc=0.57343 
[150/150]: loss=0.65134 val_loss=0.68726 acc=0.64554 val_acc=0.57343 


In [14]:
nn.summary()

|Dense(30, 30, tanh)              	|	930
|Dense(30, 65, elu)              	|	2015
|Dense(65, 64, leaky_relu)             	|	4224
|Dense(64, 1, sigmoid)               	|	65
Total number of parameters:		 7234
Total number of trainable params:	 7234


In [15]:
accuracy()(y_train, nn.predict(X_train.values))

0.6455399061032864

In [16]:
nn.fit(X_train.iloc[0, :].values.reshape(1, -1), y_train[0, :])

[1/1]: loss=0.47432 


In [17]:
# from keras.layers import Dense
# from keras.models import Sequential

# nk = Sequential()
# nk.add(Dense(10, activation='relu'))
# nk.add(Dense(1, activation='sigmoid'))
# nk.compile('sgd', loss='binary_crossentropy')
# nk.fit(X_train.values, y_train, epochs=10)